In [13]:
import pandas as pd
from tqdm.notebook import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from lightgbm import LGBMClassifier

In [24]:
# loading dataframe
df = pd.read_csv('./newly_reduced_bungae_df.csv', lineterminator='\n') # 이걸로 오류 해결
df['id'] = df['id'].astype(str)
df['cat_id'] = df['cat_id'].astype(str)
df['maincat_id'] = df['maincat_id'].astype(str)
df['midcat_id'] = df['midcat_id'].astype(str)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 109248 entries, 0 to 109247
Data columns (total 7 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   id          109248 non-null  object
 1   name        109248 non-null  object
 2   info        108882 non-null  object
 3   cat_id      109248 non-null  object
 4   tag         99639 non-null   object
 5   maincat_id  109248 non-null  object
 6   midcat_id   109248 non-null  object
dtypes: object(7)
memory usage: 5.8+ MB


In [31]:
# spliting data

def split_train_test_data(df):
    # -- train, test split --
    train_x, test_x, train_y, test_y = train_test_split(df['name'],
                                                        df['maincat_id'].values, # main  
#                                                         df['midcat_id'].values, # mid  
#                                                         df['cat_id'].values, # sub  
                                                        random_state=42, 
                                                        test_size=.2)

    return train_x, test_x, train_y, test_y

train_x, test_x, train_y, test_y = split_train_test_data(df)

In [32]:
tfidf_vect = TfidfVectorizer(ngram_range=(1,2), max_df=300, min_df=5)
# -- x_train vectorization --
X_train_cnt_vect = tfidf_vect.fit_transform(train_x)
# -- x_test vectorization --
X_test_cnt_vect = tfidf_vect.transform(test_x)

In [17]:
# -- logistic regression --
lr_clf = LogisticRegression(solver='liblinear', n_jobs=-1)
lr_clf.fit(X_train_cnt_vect , train_y)
preds = lr_clf.predict(X_test_cnt_vect)
print('TF-IDF Vectorized Logistic Regression 의 예측 정확도는 {0:.3f}'.format(accuracy_score(test_y,preds)))


/Users/wonbinchoi/opt/anaconda3/envs/finalPJT_v2/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1211: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 8.
  warnings.warn(


TF-IDF Vectorized Logistic Regression 의 예측 정확도는 0.751


In [33]:
# -- lightGBM --
lgbm_clf = LGBMClassifier(n_jobs=-1)
lgbm_clf.fit(X_train_cnt_vect, train_y)
preds = lgbm_clf.predict(X_test_cnt_vect)
print('lightGBM Vectorized Logistic Regression 의 예측 정확도는 {0:.3f}'.format(accuracy_score(test_y,preds)))

lightGBM Vectorized Logistic Regression 의 예측 정확도는 0.656
